## Introduction
Instances of `go.Figure` and `go.FigureWidget` can be displayed in a notebook in different ways, for example:
* `go.Figure`
  - Calling `fig.show()`
  - Returning `fig` as cell output
* `go.FigureWidget`
  - Calling `widget.show()`
  - Returning `widget` as cell output
  - Calling `display(widget)` using `display` imported from `IPython.display`

This notebook experiments with these, trying to determine two things:
* Which of these allow subsequent edits to traces on subplots of the figure
* Which of these produce figures that persist beyond the current notebook session
  * In VSCode, a good test for this is to select `Developer: Reload Window` from the command palette

The approach that was decided on is as follows:
* Use `display(widget)` to allow live updates and user interaction via click events
* Call `widget.show()`, which internally calls `go.Figure.show()` is a static rendering is required for persistence beyond the current notebook session

## Imports

In [1]:
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from IPython.display import display

## Experiments

### `fig.show()`
* This produces a static HTML
* The figure persists beyond the current notebook session
* Updates to the figure are not applied to instances already being displayed
* Calling `fig.show()` again after an update displays a new figure that includes the updates

Side note: `renderer='notebook_connected'` causes the `plotly` JS library to be imported into the notebook DOM using a CDN, which prevents adding its full contents (~4MB) to the notebook

In [2]:
fig = make_subplots(rows=2, cols=2, subplot_titles=["Plot 1", "Plot 2", "Plot 3", "Plot 4"])
fig.add_trace(go.Scatter(x=[1, 2, 3], y=[4, 5, 6], name="Trace 1"), row=1, col=1)
fig.add_trace(go.Bar(x=["A", "B", "C"], y=[7, 8, 9], name="Trace 2"), row=1, col=2)
fig.add_trace(go.Bar(x=["X", "Y", "Z"], y=[10, 20, 30], name="Trace 3"), row=2, col=1)
fig.add_trace(go.Scatter(x=[3, 4, 5], y=[9, 7, 5], name="Trace 4"), row=2, col=2)
fig.update_layout(height=300, width=600, margin=dict(l=8, r=8, t=24, b=8))
fig.show(renderer='notebook_connected')

In [3]:
fig.data[1].update(y=[9,8,7])  # Not updated on the static display above

Bar({
    'name': 'Trace 2', 'x': ['A', 'B', 'C'], 'xaxis': 'x2', 'y': [9, 8, 7], 'yaxis': 'y2'
})

In [4]:
fig.show(renderer='notebook_connected') # Now shows figure with updated data

### `fig`
* This relies on the `_repr_html_` and `_repr_mimebundle_` implementations of the `go.Figure` object
* On my some setups, this also inserts the full contents (~4MB) of the `plotly` JS library into the notebook DOM by default
* To control this:
  * Call `fig.show()` and explicitly provide a `renderer`
  * Set `pio.renderers.default` before returning the figure
  * The `notebook_connected` renderer imports the `plotly` JS library via a CDN, with two implications:
    * The notebook size does not grow by 4MB
    * The figure requires an internet connection to display correctly 
* To inspect the notebook DOM:
  * Save this notebook 
  * Create a copy of this notebook with the extension renamed to ".json"
  * Inspect the ".json" file 

In [5]:
pio.renderers.default

'plotly_mimetype+notebook'

In [6]:
pio.renderers.default = 'notebook_connected'
fig

### `widget.show()`
* This just calls `fig.show()` on the underlying `go.Figure()` object, resulting in the same static output as earlier
* This is very useful for creating snapshots of an interactive widget that will persist beyond the current notebook session

In [7]:
widget = go.FigureWidget(make_subplots(rows=2, cols=2, subplot_titles=["Plot 1", "Plot 2", "Plot 3", "Plot 4"]))
widget.add_trace(go.Scatter(x=[1, 2, 3], y=[4, 5, 6], name="Trace 1"), row=1, col=1)
widget.add_trace(go.Bar(x=["A", "B", "C"], y=[7, 8, 9], name="Trace 2"), row=1, col=2)
widget.add_trace(go.Bar(x=["X", "Y", "Z"], y=[10, 20, 30], name="Trace 3"), row=2, col=1)
widget.add_trace(go.Scatter(x=[3, 4, 5], y=[9, 7, 5], name="Trace 4"), row=2, col=2)
widget.update_layout(height=300, width=600, margin=dict(l=8, r=8, t=24, b=8))
widget.show(renderer='notebook_connected')

In [8]:
widget.data[1].update(y=[9,8,7])  # Not updated on the static display above

Bar({
    'name': 'Trace 2',
    'uid': '5106b12d-fb74-4d7b-bd90-6ef6e88f5bdf',
    'x': [A, B, C],
    'xaxis': 'x2',
    'y': [9, 8, 7],
    'yaxis': 'y2'
})

In [9]:
widget.show(renderer='notebook_connected') # Now shows updated widget

### `widget`
* This relies on the `_repr_mimebundle_` implementation of the `go.FigureWidget` object, which just inserts a `ipywidgets` model ID into the notebook DOM
* The contents of the plotly JS library is not inserted into the DOM
* Updates to the data are immediately applied to the widget currently being displayed
* The figure does not persist beyond the current notebook session

In [10]:
widget

FigureWidget({
    'data': [{'name': 'Trace 1',
              'type': 'scatter',
              'uid': '3f004ac4-ee96-4a5d-acca-f5bc127e2636',
              'x': [1, 2, 3],
              'xaxis': 'x',
              'y': [4, 5, 6],
              'yaxis': 'y'},
             {'name': 'Trace 2',
              'type': 'bar',
              'uid': '5106b12d-fb74-4d7b-bd90-6ef6e88f5bdf',
              'x': [A, B, C],
              'xaxis': 'x2',
              'y': [9, 8, 7],
              'yaxis': 'y2'},
             {'name': 'Trace 3',
              'type': 'bar',
              'uid': '518efd3b-8a0b-43c4-83d7-81fc081f6975',
              'x': [X, Y, Z],
              'xaxis': 'x3',
              'y': [10, 20, 30],
              'yaxis': 'y3'},
             {'name': 'Trace 4',
              'type': 'scatter',
              'uid': 'a9180d7f-a727-4019-a22e-54fbf181d515',
              'x': [3, 4, 5],
              'xaxis': 'x4',
              'y': [9, 7, 5],
              'yaxis': 'y4'}],
    'la

In [11]:
widget.data[2].update(y=[30,20,10])  # Updated dynamically above

Bar({
    'name': 'Trace 3',
    'uid': '518efd3b-8a0b-43c4-83d7-81fc081f6975',
    'x': [X, Y, Z],
    'xaxis': 'x3',
    'y': [30, 20, 10],
    'yaxis': 'y3'
})

### `display(widget)`
* This produces exactly the same result as returning `widget` as the cell output
  * The even contains the same `ipywidgets` model ID
  * This can be inspected by saving, copying and renaming the notebook to have a ":json" extension, which is then parsed nicely in most IDEs
* Subsequently modifying the data updates both this widget and the previous one with the same ID at the same time
* In fact, `display(x)` is a common way to add `x` to the output of the current cell

In [12]:
display(widget)

FigureWidget({
    'data': [{'name': 'Trace 1',
              'type': 'scatter',
              'uid': '3f004ac4-ee96-4a5d-acca-f5bc127e2636',
              'x': [1, 2, 3],
              'xaxis': 'x',
              'y': [4, 5, 6],
              'yaxis': 'y'},
             {'name': 'Trace 2',
              'type': 'bar',
              'uid': '5106b12d-fb74-4d7b-bd90-6ef6e88f5bdf',
              'x': [A, B, C],
              'xaxis': 'x2',
              'y': [9, 8, 7],
              'yaxis': 'y2'},
             {'name': 'Trace 3',
              'type': 'bar',
              'uid': '518efd3b-8a0b-43c4-83d7-81fc081f6975',
              'x': [X, Y, Z],
              'xaxis': 'x3',
              'y': [30, 20, 10],
              'yaxis': 'y3'},
             {'name': 'Trace 4',
              'type': 'scatter',
              'uid': 'a9180d7f-a727-4019-a22e-54fbf181d515',
              'x': [3, 4, 5],
              'xaxis': 'x4',
              'y': [9, 7, 5],
              'yaxis': 'y4'}],
    'la

In [13]:
widget.data[0].update(y=[4,5,4])  # Updated dynamically above

Scatter({
    'name': 'Trace 1',
    'uid': '3f004ac4-ee96-4a5d-acca-f5bc127e2636',
    'x': [1, 2, 3],
    'xaxis': 'x',
    'y': [4, 5, 4],
    'yaxis': 'y'
})